# pip e imports

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from scipy.sparse import coo_matrix, csr_matrix
import requests
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

#  Json

In [2]:
df_business = pd.read_json('yelp_academic_dataset_business.json', lines=True)

# Business

In [3]:
# Contar cuántos restaurantes tienen más de 100 reseñas
num_restaurants_over_100 = df_business[df_business['review_count'] > 70].shape[0]

# Mostrar el resultado
print(f"Número de restaurantes con más de 100 reseñas: {num_restaurants_over_100}")
df_business_fil=df_business[df_business['review_count'] > 70]

Número de restaurantes con más de 100 reseñas: 20997


# alternativa por si no corre con el parquet: correr el de abajo en vez del de arriba

In [ ]:
# # Contar cuántos restaurantes tienen más de 100 reseñas
# num_restaurants_over_100 = df_business[df_business['review_count'] > 400].shape[0]

# # Mostrar el resultado
# print(f"Número de restaurantes con más de 100 reseñas: {num_restaurants_over_100}")
# df_business_fil=df_business[df_business['review_count'] > 400]

Número de restaurantes con más de 100 reseñas: 2010


In [4]:
df_business_fil_fil = df_business_fil.dropna(subset=['business_id'])

In [6]:
pip install pyarrow


   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB 495.5 kB/s eta 0:00:51
   ---------------------------------------- 0.2/25.2 MB 1.4 MB/s eta 0:00:18
    --------------------------------------- 0.6/25.2 MB 3.3 MB/s eta 0:00:08
   - -------------------------------------- 1.2/25.2 MB 5.6 MB/s eta 0:00:05
   --- ------------------------------------ 2.5/25.2 MB 9.9 MB/s eta 0:00:03
   ---- ----------------------------------- 2.8/25.2 MB 10.7 MB/s eta 0:00:03
   ----- ---------------------------------- 3.7/25.2 MB 10.4 MB/s eta 0:00:03
   ------- -------------------------------- 5.0/25.2 MB 12.3 MB/s eta 0:00:02
   --------- ------------------------------ 6.1/25.2 MB 12.9 MB/s eta 0:00:02
   ----------- ---------------------------- 7.1/25.2 MB 13.8 MB/s eta 0:00:02
   ------------- -------------------------- 8.3/25.2 MB 15.1 MB/s eta 0:00:02
   -

In [8]:
df_business_fil.to_parquet('mi_archivo.parquet', engine='pyarrow')

In [9]:
df_business_fil.to_parquet('mi_archivo1.parquet', engine='pyarrow', compression='snappy')

In [ ]:
df_business_fil_fil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [89]:
expanded_cols = pd.json_normalize(df_business_fil_fil['hours'])
df_business_fil_fil = pd.concat([df_business_fil_fil.drop(columns=['hours']), expanded_cols], axis=1)

In [90]:
expanded_cols = pd.json_normalize(df_business_fil_fil['attributes'])
df_business_fil_fil = pd.concat([df_business_fil_fil.drop(columns=['attributes']), expanded_cols], axis=1)

In [ ]:
df_business_fil_fil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 58 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   business_id                 150346 non-null  object 
 1   name                        150346 non-null  object 
 2   address                     150346 non-null  object 
 3   city                        150346 non-null  object 
 4   state                       150346 non-null  object 
 5   postal_code                 150346 non-null  object 
 6   latitude                    150346 non-null  float64
 7   longitude                   150346 non-null  float64
 8   stars                       150346 non-null  float64
 9   review_count                150346 non-null  int64  
 10  is_open                     150346 non-null  int64  
 11  categories                  150243 non-null  object 
 12  Monday                      114474 non-null  object 
 13  Tuesday       

# Modelo simple 1: en base a lat y long, te recomienda los de mas estrellas y mas review_counts dentro de los que estan cerca de la ubicacion.

In [83]:
pip install folium

   ---------------------------------------- 0.0/108.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/108.9 kB ? eta -:--:--
   ----------------------------------- -- 102.4/108.9 kB 845.5 kB/s eta 0:00:01
   -------------------------------------- 108.9/108.9 kB 789.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
   ---------------------------------------- 85.1/85.1 kB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [84]:
pip install config

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import hvplot.pandas

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#from config import MAPBOX_API_KEY

In [11]:
df_business_fil_fil = pd.read_parquet('mi_archivo.parquet', engine='pyarrow')

In [12]:
df_business_fil_fil = pd.read_parquet('mi_archivo1.parquet', engine='pyarrow')

In [94]:
df_business_fil_fil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54369 entries, 3 to 38948
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 20997 non-null  object 
 1   name                        20997 non-null  object 
 2   address                     20997 non-null  object 
 3   city                        20997 non-null  object 
 4   state                       20997 non-null  object 
 5   postal_code                 20997 non-null  object 
 6   latitude                    20997 non-null  float64
 7   longitude                   20997 non-null  float64
 8   stars                       20997 non-null  float64
 9   review_count                20997 non-null  float64
 10  is_open                     20997 non-null  float64
 11  categories                  20997 non-null  object 
 12  Monday                      17824 non-null  object 
 13  Tuesday                     18714 no

In [95]:
top_restaurants_df = df_business_fil_fil.sort_values(by=["stars","review_count"], ascending=False)[:20]

In [102]:
df_business_fil_fil['latitude'].isna().sum()

33372

In [101]:
print(f"Número total de filas en el DataFrame: {df_business_fil_fil.shape[0]}")


Número total de filas en el DataFrame: 54369


In [103]:
df_business_location = df_business_fil_fil.dropna(subset=['latitude', 'longitude'])

In [104]:
#Elbow method to determine the number of K in Kmeans Clustering
coords = df_business_location[["latitude", "longitude"]]

inertia = []
K = range(1,25)

# Calculate the inertia for the range of K values
for k in K:
    kmeansModel = KMeans(n_clusters=k)
    kmeansModel = kmeansModel.fit(coords)
    inertia.append(kmeansModel.inertia_)

In [111]:
import matplotlib.pyplot as plt

# Graficar el método del codo
plt.figure(figsize=(10, 6))
plt.plot(K, inertia, 'bo-', color='blue', markersize=8)
plt.xlabel('Número de clústeres (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para determinar el número óptimo de clústeres')
plt.grid(True)

# Guardar la gráfica en un archivo
plt.savefig('metodo_del_codo.png', dpi=300)

# Opcionalmente, cerrar la figura para liberar memoria
plt.close()


In [112]:
import numpy as np

# Calcular la tasa de cambio relativa de la inercia
rate_of_change = np.diff(inertia) / inertia[:-1]

# Crear un DataFrame para mostrar las tasas de cambio
import pandas as pd
resultados = pd.DataFrame({
    'k': list(K)[:-1],  # Hasta el penúltimo porque np.diff reduce el tamaño
    'inercia': inertia[:-1],
    'tasa_de_cambio': rate_of_change
})

# Imprimir los resultados ordenados
print(resultados)

# Encontrar el índice con la menor reducción significativa
k_optimo = resultados['tasa_de_cambio'].idxmin() + 1  # +1 porque la lista de índices empieza en 0
print(f"El número óptimo de clústeres (k) podría ser: {k_optimo}")

     k     inercia    tasa_de_cambio
0    1  5.295160e+06    -0.771681   
1    2  1.208983e+06    -0.539731   
2    3  5.564574e+05    -0.457231   
3    4  3.020279e+05    -0.352344   
4    5  1.956101e+05    -0.387001   
5    6  1.199087e+05    -0.488925   
6    7  6.128239e+04    -0.333721   
7    8  4.083114e+04    -0.422177   
8    9  2.359318e+04    -0.508375   
9   10  1.159900e+04    -0.949567   
10  11  5.849710e+02    -0.222086   
11  12  4.550570e+02    -0.176186   
12  13  3.748824e+02    -0.141745   
13  14  3.217445e+02    -0.102916   
14  15  2.886318e+02    -0.060446   
15  16  2.711853e+02    -0.089947   
16  17  2.467929e+02    -0.035243   
17  18  2.380951e+02    -0.075255   
18  19  2.201772e+02    -0.038637   
19  20  2.116701e+02    -0.063383   
20  21  1.982537e+02    -0.062647   
21  22  1.858337e+02    -0.018712   
22  23  1.823563e+02    -0.060920   
El número óptimo de clústeres (k) podría ser: 10


In [113]:
kmeans = KMeans(n_clusters= 10, init='k-means++')
kmeans.fit(coords)
y= kmeans.labels_

In [114]:
df_business_location["cluster"]= kmeans.predict(df_business_location[['latitude', 'longitude']])

In [116]:
top_restaurants = df_business_location.sort_values(by=['stars','review_count'],ascending=False)


In [122]:
top_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20997 entries, 12307 to 131622
Data columns (total 59 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 20997 non-null  object 
 1   name                        20997 non-null  object 
 2   address                     20997 non-null  object 
 3   city                        20997 non-null  object 
 4   state                       20997 non-null  object 
 5   postal_code                 20997 non-null  object 
 6   latitude                    20997 non-null  float64
 7   longitude                   20997 non-null  float64
 8   stars                       20997 non-null  float64
 9   review_count                20997 non-null  float64
 10  is_open                     20997 non-null  float64
 11  categories                  20997 non-null  object 
 12  Monday                      2590 non-null   object 
 13  Tuesday                     271

In [124]:
def recommend_restaurants (df,latitude, longitude):
    # Predict the cluster for longitude and laltiude provided
    cluster = kmeans.predict(np.array([latitude, longitude]).reshape(1, -1))[0]
    print(cluster)
    #Get the best reataurant in this cluster
    return df[df["cluster"]==cluster].iloc[0:5][['name', 'latitude', 'longitude','stars','categories','review_count','business_id']]

In [125]:
recommend_restaurants(top_restaurants,39.97165, -75.12917)

2


,name,latitude,longitude,stars,categories,review_count,business_id
104287,SUGARED + BRONZED,39.948570,-75.160072,5.0,"Beauty & Spas, Shopping, Waxing, Cosmetics & Beauty Supply, Day Spas, Tanning, Spray Tanning, Hair Removal, Sugaring",513.0,l_7TW_Ix58-QvhQgpJi_Xw
84744,JB Plumbing and Heating,39.955929,-75.157457,5.0,"Plumbing, Water Heater Installation/Repair, Excavation Services, Contractors, Heating & Air Conditioning/HVAC, Home Services",398.0,u3zqvp4BYUjzJD7tzx3Jbg
7094,The Sweet Life Bakeshop,39.942203,-75.155223,5.0,"Food, Cupcakes, Desserts, Bakeries",316.0,qjGS_7iaQDpbVhS6W8qkHQ
102283,Bobby Mack & Co Hair Studio,39.944711,-75.174965,5.0,"Hair Salons, Men's Hair Salons, Hair Stylists, Beauty & Spas",291.0,Og4z8nB4ZMZs3oHkVhB_pA
67317,ICI Macarons & Cafe,39.951910,-75.144381,5.0,"Desserts, Bakeries, Food, Coffee & Tea, Specialty Food, Macarons",276.0,-fs09akgCKv5rTTy7iUHUg
